In [1]:
from google.cloud import storage
import pandas as pd ## for dataset and eda
import numpy as np ## for eda
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# %matplotlib inline
print("All good")

All good


In [2]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
# from transformers import DistilBertModel, DistilBertTokenizer # do not forget to conda install transformers
from transformers import *
print("All packages are imported")

All packages are imported


In [3]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"GPU usage is set up: {device}")

GPU usage is set up: cuda


In [4]:
cuda.is_available()

True

In [5]:
bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

print(bucket)
print('Great, we now have access to our first bucket on google cloud storage where we put our data')

<Bucket: firstprojectdl>
Great, we now have access to our first bucket on google cloud storage where we put our data


In [6]:
from google.cloud import storage
import pandas as pd

bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "data/hw2/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')


for blob in blobs:
    print(blob.name)
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        print(file_name)
print(f"we imported the {file_name} successfully")

data/hw2/
data/hw2/nodeid2paperid.csv
nodeid2paperid.csv
data/hw2/sample.csv
sample.csv
data/hw2/testData.csv
testData.csv
data/hw2/textData.csv
textData.csv
data/hw2/trainData.csv
trainData.csv
we imported the trainData.csv successfully


In [7]:
nodeid2paperid = pd.read_csv('nodeid2paperid.csv')
nodeid2paperid.head()
nodeid2paperid.rename(columns={'node idx': 'id'}, inplace=True)
nodeid2paperid.head()

,id,paper id
0,104447,630234
1,15858,803423
2,107156,1102481
3,82077,1810480
4,42436,2131697


In [8]:
test = pd.read_csv('testData.csv')
test.head()

,id
0,137832
1,137833
2,137834
3,137836
4,137837


In [9]:
test.shape

(13718, 1)

In [10]:
text = pd.read_csv('textData.csv')
text.head()

,paper id,title,abstract
0,630234,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
1,803423,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
2,1102481,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...
3,1810480,cryptographic hardening of d sequences,This paper shows how a one-way mapping using m...
4,2131697,gesture based continuous authentication for we...,We study the feasibility of touch gesture beha...


In [11]:
text.shape

(73718, 3)

In [12]:
train = pd.read_csv('trainData.csv')
train.head()

,label,id
0,4,0
1,5,1
2,8,3
3,6,6
4,4,7


In [13]:
train.shape

(60000, 2)

In [14]:
sample = pd.read_csv('sample.csv')
sample.head()

,id,label
0,137832,0
1,137833,0
2,137834,0
3,137836,0
4,137837,0


In [15]:
sample.shape

(13718, 2)

In [16]:
## Let us merge text in train and test
# train
trainData = pd.merge(train, nodeid2paperid, on='id', how='inner')

In [17]:
trainData = pd.merge(trainData, text, on ='paper id', how='inner')

In [18]:
trainData.head()

,label,id,paper id,title,abstract
0,4,0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes..."
1,5,1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...
2,8,3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...
3,6,6,1444859417,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...
4,4,7,1483430697,information theoretic authentication and secre...,"In the splitting model, information theoretic ..."


In [19]:
trainData.shape

(60000, 5)

In [20]:
# test
testData = pd.merge(test, nodeid2paperid, on='id', how='inner')

In [21]:
testData.head()

,id,paper id
0,137832,2403725649
1,137833,2404740077
2,137834,2407125866
3,137836,2408327416
4,137837,2412021890


In [22]:
testData = pd.merge(testData, text, on ='paper id', how='inner')

In [23]:
testData.head()

,id,paper id,title,abstract
0,137832,2403725649,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,2404740077,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,2407125866,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,2408327416,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,2412021890,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [24]:
testData.shape

(13718, 4)

In [25]:
# # Removing unwanted columns and only leaving title and abstract of the scientific article and the category which will be the label
trainData = trainData[['id','title', 'abstract', 'label']]
labelListData = list(trainData['label'].unique())
trainData.head()

,id,title,abstract,label
0,0,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",4
1,1,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,5
2,3,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,8
3,6,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...,6
4,7,information theoretic authentication and secre...,"In the splitting model, information theoretic ...",4


In [26]:
testData = testData[['id','title', 'abstract']]
testData.head()

,id,title,abstract
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [27]:
# Add a comma at the end of the titles
trainData['title'] = trainData['title'].apply(lambda x: x + '.' if x[:-1] != '.' else x)

In [28]:
## merge the title and abstract columns together 
trainData['description'] = trainData['title'].str.cat(trainData['abstract'],sep=" ")
trainData['description'] = trainData['description'].apply(lambda x: x[0:511])
## trainData['description'] = trainData['title']

In [29]:
## add a period at the end of the truncated 

trainData['description'] = trainData['description'].apply(lambda x: x + '.' if x[-1] != '.' else x)

# Remove stop words in description and title ## Actually mention why you may not want to remove stop words: https://stackoverflow.com/questions/63633534/is-it-necessary-to-do-stopwords-removal-stemming-lemmatization-for-text-classif

In [30]:
trainData = trainData[['description', 'label']]

In [31]:
trainData.isna().sum()

description    0
label          0
dtype: int64

In [32]:
len(labelListData) # 20 labels and not 40

20

In [33]:
labelListData

[4, 5, 8, 6, 3, 16, 19, 14, 10, 0, 2, 18, 9, 13, 11, 1, 7, 15, 17, 12]

In [34]:
# biggest text in column
trainData['description'].apply(lambda x: len(x)).max()

512

In [35]:
trainData

,description,label
0,evasion attacks against machine learning at te...,4
1,how hard is computing parity with noisy commun...,5
2,a promise theory perspective on data networks....,8
3,webvrgis based city bigdata 3d visualization a...,6
4,information theoretic authentication and secre...,4
...,...,...
59995,incentivizing users of data centers participat...,5
59996,semantic change detection with hypermaps. Chan...,16
59997,computing with polynomial ordinary differentia...,9
59998,on energy efficiency in wireless networks a ga...,8


In [36]:
# Add a comma at the end of the titles
testData['title'] = testData['title'].apply(lambda x: x + '.' if x[:-1] != '.' else x)
testData['description'] = testData['title'].str.cat(testData['abstract'],sep=" ")
testData['description'] = testData['description'].apply(lambda x: x[0:511])

In [37]:
## add a period at the end of the truncated 

testData['description'] = testData['description'].apply(lambda x: x + '.' if x[-1] != '.' else x)

In [38]:
testData

,id,title,abstract,description
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal...",patchlift fast and exact computation of patch ...
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...,the unreasonable effectiveness of address clus...
2,137834,end to end goal driven web navigation.,We propose a goal-driven web navigation as a b...,end to end goal driven web navigation. We prop...
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...,complexity measures for map reduce and compari...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...,a parallel implementation of the ensemble kalm...
...,...,...,...,...
13713,169336,confidence guided stereo 3d object detection w...,Accurate and reliable 3D object detection is v...,confidence guided stereo 3d object detection w...
13714,169338,sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,sentinet detecting localized universal attacks...
13715,169340,learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",learning compositional rules via neural progra...
13716,169341,certified defenses for adversarial patches.,Adversarial patch attacks are among one of the...,certified defenses for adversarial patches. Ad...


In [39]:
testData = testData[['description']]

In [40]:
testData

,description
0,patchlift fast and exact computation of patch ...
1,the unreasonable effectiveness of address clus...
2,end to end goal driven web navigation. We prop...
3,complexity measures for map reduce and compari...
4,a parallel implementation of the ensemble kalm...
...,...
13713,confidence guided stereo 3d object detection w...
13714,sentinet detecting localized universal attacks...
13715,learning compositional rules via neural progra...
13716,certified defenses for adversarial patches. Ad...


In [41]:
# biggest text in column
testData['description'].apply(lambda x: len(x)).max()

512

In [42]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 3e-05
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')

In [43]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [44]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset= trainData.sample(frac=train_size,random_state=200)
val_dataset= trainData.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(val_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
validation_set = Triage(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (60000, 2)
TRAIN Dataset: (48000, 2)
VALIDATION Dataset: (12000, 2)


In [45]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)

In [46]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class textClassModel(torch.nn.Module):
    def __init__(self):
        super(textClassModel, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 20)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [47]:
model = textClassModel()
model.to(device)

textClassModel(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [48]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [49]:
# Function to calcuate the accuracy of the model

def calculate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [50]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
            print(f"Training Accuracy per 1000 steps: {accu_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train(epoch)

  0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 1000 steps: 3.004389524459839
Training Accuracy per 1000 steps: 6.25


In [447]:
torch.save(model.state_dict(), 'textClassModel.pt')

In [448]:
def valid(epoch):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    n_correct = 0 
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%1000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 1000 steps: {loss_step}")
                print(f"Validation Accuracy per 1000 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [449]:
for epoch in tqdm(range(EPOCHS)):
    acc = valid(epoch)
    print("Accuracy on test data = %0.2f%%" % acc)

  0%|          | 0/10 [00:00<?, ?it/s]

Validation Loss per 1000 steps: 0.046036601066589355
Validation Accuracy per 1000 steps: 100.0
Validation Loss per 1000 steps: 1.027636722846703
Validation Accuracy per 1000 steps: 82.29270729270729
Validation Loss per 1000 steps: 1.030877064073878
Validation Accuracy per 1000 steps: 82.23388305847077


 10%|█         | 1/10 [03:30<31:36, 210.76s/it]

Validation Loss Epoch: 1.0323521004319192
Validation Accuracy Epoch: 82.23333333333333
Accuracy on test data = 82.23%
Validation Loss per 1000 steps: 0.17684710025787354
Validation Accuracy per 1000 steps: 75.0
Validation Loss per 1000 steps: 1.0689709882933895
Validation Accuracy per 1000 steps: 81.51848151848152
Validation Loss per 1000 steps: 1.0376076216491563
Validation Accuracy per 1000 steps: 81.92153923038481


 20%|██        | 2/10 [07:09<28:24, 213.03s/it]

...

Validation Loss Epoch: 1.0323521006802718
Validation Accuracy Epoch: 82.23333333333333
Accuracy on test data = 82.23%
Validation Loss per 1000 steps: 0.014478325843811035
Validation Accuracy per 1000 steps: 100.0
Validation Loss per 1000 steps: 0.948469355479106
Validation Accuracy per 1000 steps: 83.61638361638362
Validation Loss per 1000 steps: 1.0052590153772554
Validation Accuracy per 1000 steps: 82.72113943028486


 90%|█████████ | 9/10 [32:36<03:37, 217.86s/it]

Validation Loss Epoch: 1.0323521007299423
Validation Accuracy Epoch: 82.23333333333333
Accuracy on test data = 82.23%
Validation Loss per 1000 steps: 1.9553031921386719
Validation Accuracy per 1000 steps: 75.0
Validation Loss per 1000 steps: 0.9880386091374256
Validation Accuracy per 1000 steps: 82.71728271728271
Validation Loss per 1000 steps: 1.052372534057011
Validation Accuracy per 1000 steps: 82.00899550224888


100%|██████████| 10/10 [36:14<00:00, 217.47s/it]

Validation Loss Epoch: 1.0323520993590356
Validation Accuracy Epoch: 82.23333333333333
Accuracy on test data = 82.23%


In [450]:
class TriageTest(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        } 
    
    def __len__(self):
        return self.len

In [451]:
testSet = TriageTest(testData, tokenizer, MAX_LEN)

In [452]:
test_params = {'batch_size': 8,
                'shuffle': False,
                'num_workers': 0
                }


testing_loader = DataLoader(testSet, **test_params)

In [453]:
predictions = []
with torch.no_grad():
    for _, data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask).squeeze()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx)



1715it [03:58,  7.19it/s]


In [454]:
print(predictions)
listOfList = [list(x.cpu().numpy()) for x in predictions]
result = [item for sublist in listOfList for item in sublist]

[tensor([16,  8, 10,  5,  5, 15,  5,  5], device='cuda:0'), tensor([ 8,  8, 16,  0,  9,  0, 16, 10], device='cuda:0'), tensor([16, 16,  4, 16, 16, 16, 19, 19], device='cuda:0'), tensor([11,  8,  8, 16, 16, 19, 16, 16], device='cuda:0'), tensor([ 3, 16, 16, 16,  8, 19, 10, 19], device='cuda:0'), tensor([ 7, 16,  8,  8, 14, 16,  8, 16], device='cuda:0'), tensor([ 6,  7,  5, 16, 16,  4, 13, 17], device='cuda:0'), tensor([19,  8,  4,  3, 16, 16, 19, 16], device='cuda:0'), tensor([16,  4, 16,  2, 16,  5,  2, 16], device='cuda:0'), tensor([ 2,  9, 16,  4,  4, 10, 19, 10], device='cuda:0'), tensor([10, 16, 19, 16, 16,  9,  4, 16], device='cuda:0'), tensor([19, 16,  2, 13,  3,  8, 16,  3], device='cuda:0'), tensor([16,  5, 19,  4, 16, 16,  8, 16], device='cuda:0'), tensor([16, 10,  4,  2, 16,  4,  6, 16], device='cuda:0'), tensor([ 2, 16, 16, 16, 16, 10, 16,  8], device='cuda:0'), tensor([ 2, 16, 16, 16, 16, 16, 16,  5], device='cuda:0'), tensor([ 5, 16, 10,  5, 16, 14,  5, 16], device='cuda:0

In [457]:
len(result)
result

[16,
 8,
 10,
 5,
 5,
 15,
 5,
 5,
 8,
 8,
 16,
 0,
 9,
 0,
 16,
 10,
 16,
 16,
 4,
 16,
 16,
 16,
 19,
 19,
 11,
 8,
 8,
 16,
 16,
 19,
 16,
 16,
 3,
 16,
 16,
 16,
 8,
 19,
 10,
 19,
 7,
 16,
 8,
 8,
 14,
 16,
 8,
 16,
 6,
 7,
 5,
 16,
 16,
 4,
 13,
 17,
 19,
 8,
 4,
 3,
 16,
 16,
 19,
 16,
 16,
 4,
 16,
 2,
 16,
 5,
 2,
 16,
 2,
 9,
 16,
 4,
 4,
 10,
 19,
 10,
 10,
 16,
 19,
 16,
 16,
 9,
 4,
 16,
 19,
 16,
 2,
 13,
 3,
 8,
 16,
 3,
 16,
 5,
 19,
 4,
 16,
 16,
 8,
 16,
 16,
 10,
 4,
 2,
 16,
 4,
 6,
 16,
 2,
 16,
 16,
 16,
 16,
 10,
 16,
 8,
 2,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 5,
 16,
 10,
 5,
 16,
 14,
 5,
 16,
 8,
 3,
 16,
 5,
 16,
 16,
 15,
 16,
 2,
 10,
 16,
 16,
 16,
 16,
 16,
 18,
 0,
 18,
 16,
 8,
 2,
 10,
 10,
 16,
 8,
 16,
 0,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 8,
 8,
 8,
 5,
 3,
 16,
 5,
 16,
 16,
 16,
 16,
 16,
 16,
 7,
 16,
 16,
 16,
 16,
 13,
 3,
 16,
 8,
 4,
 16,
 16,
 16,
 19,
 16,
 2,
 16,
 16,
 16,
 16,
 16,
 8,
 16,
 8,
 19,
 16,
 16,
 4,
 16,
 6,
 19,
 9,
 16,


In [458]:
submission = sample
submission['label'] = result

In [459]:
submission

,id,label
0,137832,16
1,137833,8
2,137834,10
3,137836,5
4,137837,5
...,...,...
13713,169336,16
13714,169338,16
13715,169340,10
13716,169341,16


In [460]:
submission.to_csv('submission1.csv', index = False)

In [ ]:
## list of things left to experiment and fine tune
# 1. train bert with abstract only
# 2. train bert with title and abstract
# 3. train bert with batch size at 16 or 32
# 4. Tweak the max length for padding depending on the distribution length of the paragraphs
## 5. Very important: also try with 4 epochs
## Play with the drop out level at 0.2, 0.3, and 0.5
## try scibert pre trained scientific model 